In [1]:
from cvrp_algorithms import SimpleAco, AcoSigmaBestAnts, AcoImprovedPheromoneUpdate, Aco2opt, Greedy
import cvrpcases as cvrp
from array import *
import math
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.backends.backend_pdf import PdfPages

def listToString(list):
    str1 = ''   
    for ele in list:  
        str1 += str(ele)+'\n' 
    str1 = str1[:-2] 
    return str1  

def saveFile(firstLine, vector, file):
    text_file = open(file, "w+")
    text_file.write(firstLine+'\n')
    n = text_file.write(listToString(vector))
    text_file.close()

def readVector(file):
    file1 = open(file, 'r') 
    Lines = file1.readlines() 
    v = np.zeros(len(Lines)-1)
    for it in range(len(v)):
        v[it] = float(Lines[it+1])
    return v

def binarySearchAntsCount(refTime,iterations, mincount = 1,maxcount = 100):
    #print(mincount)
    #print(maxcount)
    #print("********************")
    if(mincount == maxcount):
        algorithm = SimpleAco(cvrp.load_case(r'datasets\A\A-n32-k5.vrp'), mincount, max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=0, seed = 78)
        start = time.time()
        solution, path = algorithm.compute(log_level=0)
        tim = time.time()-start
        #print(tim)
        #print("**********")
        return tim, solution, mincount
    if(maxcount - mincount == 1):
        algorithm1 = SimpleAco(cvrp.load_case(r'datasets\A\A-n32-k5.vrp'), mincount, max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=0, seed = 78)
        start1 = time.time()
        solution1, path1 = algorithm1.compute(log_level=0)
        end1 = time.time()
        algorithm2 = SimpleAco(cvrp.load_case(r'datasets\A\A-n32-k5.vrp'), maxcount, max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=0, seed = 78)
        start2 = time.time()
        solution2, path2 = algorithm1.compute(log_level=0)
        end2 = time.time()
        if(abs(refTime - end1 + start1)<=abs(refTime - end2 + start2)):
            #print(end1-start1)
            #print("**********")
            return end1-start1, solution1, mincount
        #print(end1-start1)
        #print("**********")
        return end2-start2, solution2, maxcount
    algorithm = SimpleAco(cvrp.load_case(r'datasets\A\A-n32-k5.vrp'), math.floor((maxcount + mincount)/2), max_iterations=iterations, alpha=2, beta=5, evaporation_rate=0.6, pheromone_amount=0, seed = 78)
    start = time.time()
    solution, path = algorithm.compute(log_level=0)
    time1 = time.time() - start
    if(time1>=refTime):
        return binarySearchAntsCount(refTime,iterations,mincount,math.floor((maxcount + mincount)/2))
    #print("Im here")
    #print(time1)
    #print(math.floor((maxcount + mincount)/2))
    return binarySearchAntsCount(refTime,iterations,math.floor((maxcount + mincount)/2),maxcount)

In [ ]:
cases =[
r'datasets\A\A-n32-k5.vrp',
r'datasets\A\A-n33-k5.vrp',
r'datasets\A\A-n33-k6.vrp',
r'datasets\A\A-n34-k5.vrp',
r'datasets\A\A-n36-k5.vrp',
r'datasets\A\A-n37-k5.vrp',
r'datasets\A\A-n37-k6.vrp',
r'datasets\A\A-n38-k5.vrp',
r'datasets\A\A-n39-k5.vrp',
r'datasets\A\A-n39-k6.vrp']

res = np.zeros((16,5,10))
times = np.zeros((16,5,10))
ants = np.zeros((16,5,10))
for case in range(10):
    for seed in range(0,5):
        start = time.time()
        algorithm = SimpleAco(cvrp.load_case(cases[case]), ants_count=1000, max_iterations=1, alpha=2, beta=5, evaporation_rate=1, pheromone_amount=0, seed = seed)
        res[0][seed][case], path = algorithm.compute(log_level=0)
        end = time.time()
        times[0][seed][case] = end - start
        ants[0][seed][case] = 10000
        for it in range(0,15):
            times[it+1][seed], res[it+1][seed], ants[it+1][seed] = binarySearchAntsCount(times[0][seed],it+2,1,ants[it][seed])
            print(it)

saveFile('res',path,r'results/h3/res.txt')
saveFile('ants',path,r'results/h3/ants.txt')
saveFile('times',path,r'results/h3/times.txt')